In [8]:
import pint


ureg = pint.UnitRegistry()

In [12]:
a = 2

In [13]:
sortedlist = sorted(units.items(), key=lambda s: -len(s[0]))
with open('resources/dataset/units.json', 'w', encoding='utf-8') as fp:
    json.dump(dict(sortedlist),fp,indent=4)

In [2]:
import re
import unidecode

def join_patterns(patterns, grouping=False):
    return '(' + ('?:' if not grouping else '') + '|'.join(patterns) + ')'

NUM_1_2_to_9_DICT = {
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'سو': 3,
    'چهار': 4,
    'پنج': 5,
    'شش': 6,
    'هفت': 7,
    'هشت': 8,
    'نه': 9
}

NUM_10_11_to_19_DICT = {
    'ده': 10,
    'یازده': 11,
    'دوازده': 12,
    'سیزده': 13,
    'چهارده': 14,
    'پانزده': 15,
    'شانزده': 16,
    'هفده': 17,
    'هجده': 18,
    'نوزده': 19
}

NUM_20_30_to_90_DICT = {
    'بیست': 20,
    'سی': 30,
    'چهل': 40,
    'پنجاه': 50,
    'شصت': 60,
    'هفتاد': 70,
    'هشتاد': 80,
    'نود': 90,
}

NUM_100_200_to_900_DICT = {
    'صد': 100,
    'یکصد': 100,
    'دویست': 200,
    'سیصد': 300,
    'چهارصد': 400,
    'پانصد': 500,
    'ششصد': 600,
    'هفتصد': 700,
    'هشتصد': 800,
    'نهصد': 900
}

NUM_EXTENDER_DICT = {
    'هزار': 1E3,
    'میلیون': 1E6,
    'میلیارد': 1E9
}

NUM_SPECIAL_FRACTION_DICT = {
    'نیم': 1 / 2,
    'نصف': 1 / 2,
    'ثلث': 1 / 3,
    'ربع': 1 / 4,
    'خمس': 1 / 5
}

NUM_UNDER_1000_DICT = {
    **NUM_1_2_to_9_DICT,
    **NUM_10_11_to_19_DICT,
    **NUM_20_30_to_90_DICT,
    **NUM_100_200_to_900_DICT
}

WHITE_SPACE = r'[\s\u200c]+'
CONNECTOR = rf'{WHITE_SPACE}و{WHITE_SPACE}'
NEGATE = r'منفی'
ZERO = r'صفر'
DIGITS = r'[۰۱۲۳۴۵۶۷۸۹0123456789]'
DOTS = r'[٫\.]'

# Pattern for digit-based NUM
NUM_DIGIT_BASED = rf'(?:-?{DIGITS}+(?:{DOTS}{DIGITS}+)?)'

# Pattern for NUM ∈ {1,2,...,99}
NUM_1_2_to_9 = join_patterns(NUM_1_2_to_9_DICT.keys(), True)
NUM_10_11_to_19 = join_patterns(NUM_10_11_to_19_DICT.keys(), True)
NUM_20_30_to_90 = join_patterns(NUM_20_30_to_90_DICT.keys(), True)
NUM_20_21_to_99 = rf'(?:{NUM_20_30_to_90}(?:{CONNECTOR}{NUM_1_2_to_9})?)'
NUM_1_2_to_99 = join_patterns([NUM_10_11_to_19, NUM_20_21_to_99, NUM_1_2_to_9])

# Pattern for NUM ∈ {1,2,...,999}
NUM_100_200_to_900 = join_patterns(NUM_100_200_to_900_DICT.keys(), True)
NUM_100_101_to_999 = rf'(?:{NUM_100_200_to_900}(?:{CONNECTOR}{NUM_1_2_to_99})?)'
NUM_1_2_to_999 = join_patterns([NUM_100_101_to_999, NUM_1_2_to_99, NUM_DIGIT_BASED])

# Pattern for NUM ∈ {1,2,...}
NUM_EXTENDER = join_patterns(NUM_EXTENDER_DICT.keys(), True)
NUM_1_2_to_999_EXTENDED = rf'(?:(?:{NUM_1_2_to_999}{WHITE_SPACE})?{NUM_EXTENDER}(?:{WHITE_SPACE}{NUM_EXTENDER})*)'
NUM_EXTENDED = rf'(?:{NUM_1_2_to_999_EXTENDED}(?:{CONNECTOR}{NUM_1_2_to_999_EXTENDED})*(?:{CONNECTOR}{NUM_1_2_to_999})?)'
NUM_NATURAL = join_patterns([NUM_EXTENDED, NUM_1_2_to_999])

# Pattern for NUM ∈ Z
NUM_INTEGER = join_patterns([ZERO, rf'(?:(?:({NEGATE}){WHITE_SPACE})?{NUM_NATURAL})'])

# Pattern for NUM ∈ {p/q | p,q ∈ Z}
NUM_NORMAL_FRACTION = rf'(?:{NUM_INTEGER}{WHITE_SPACE}{NUM_INTEGER}م)'
NUM_NORMAL_FRACTION_LABELED = rf'(?:(?P<numerator>{NUM_INTEGER}){WHITE_SPACE}(?P<denominator>{NUM_INTEGER})م)'
NUM_SPECIAL_FRACTION = join_patterns(NUM_SPECIAL_FRACTION_DICT.keys())
NUM_FRACTION = join_patterns([NUM_SPECIAL_FRACTION, NUM_NORMAL_FRACTION])

# Pattern for NUM ∈ {n+(p/q) | n,p,q ∈ Z}
NUM_INTEGER_PLUS_FRACTION_LABELED = rf'(?:(?P<integer_part>{NUM_INTEGER}){CONNECTOR}(?P<fraction_part>{NUM_FRACTION}))'
NUM_INTEGER_PLUS_FRACTION = rf'({NUM_INTEGER}{CONNECTOR}{NUM_FRACTION})'

# Pattern for NUM ∈ Q
NUM_RATIONAL = join_patterns([NUM_FRACTION, NUM_INTEGER_PLUS_FRACTION, NUM_INTEGER])

def get_value_NUM_DIGIT_BASED(num_string):
    return float(unidecode.unidecode(num_string))

def get_value_NUM_1_2_to_999(num_string):
    try:
        return get_value_NUM_DIGIT_BASED(num_string)
    except:
        match_groups = re.match(NUM_1_2_to_999, num_string).groups()
        return sum([NUM_UNDER_1000_DICT[key] for key in filter(None, match_groups)])

def get_value_NUM_1_2_to_999_EXTENDED(num_string):
    extender_value = 1
    num_extender_matches = re.finditer(NUM_EXTENDER, num_string)
    for match in num_extender_matches:
        extender_value *= NUM_EXTENDER_DICT[match.group()]
    try:
        num_without_extender_match = re.search(NUM_1_2_to_999, re.sub(NUM_EXTENDER, '', num_string))
        num_without_extender = get_value_NUM_1_2_to_999(num_without_extender_match.group())
    except:
        num_without_extender = 1
    return extender_value * num_without_extender

def get_value_NUM_NATURAL(num_string):
    num_1_2_to_999_extended_list = re.finditer(NUM_1_2_to_999_EXTENDED, num_string)
    num_value = 0
    for match in num_1_2_to_999_extended_list:
        num_value += get_value_NUM_1_2_to_999_EXTENDED(match.group())
    num_1_2_to_999_at_the_end = re.search(NUM_1_2_to_999, re.sub(NUM_1_2_to_999_EXTENDED, '', num_string))
    if num_1_2_to_999_at_the_end:
        num_value += get_value_NUM_1_2_to_999(num_1_2_to_999_at_the_end.group())
    return num_value

def get_value_NUM_INTEGER(num_string):
    if re.search(ZERO, num_string):
        return 0
    coefficient = 1
    if re.search(NEGATE, num_string):
        coefficient = -1
    num_natural = re.search(NUM_NATURAL, num_string).group()
    return coefficient * get_value_NUM_NATURAL(num_natural)

def get_value_NUM_FRACTION(num_string):
    match = re.search(NUM_SPECIAL_FRACTION, num_string)
    if match:
        return NUM_SPECIAL_FRACTION_DICT[match.group()]
    match = re.search(NUM_NORMAL_FRACTION_LABELED, num_string)
    numerator = get_value_NUM_INTEGER(match.group('numerator'))
    denominator = get_value_NUM_INTEGER(match.group('denominator'))
    try:
        return numerator / denominator
    except:
        return 0

def get_value_NUM_INTEGER_PLUS_FRACTION(num_string):
    match = re.search(NUM_INTEGER_PLUS_FRACTION_LABELED, num_string)
    integer_part = get_value_NUM_INTEGER(match.group('integer_part'))
    try:
        fraction_part = get_value_NUM_FRACTION(match.group('fraction_part'))
    except:
        fraction_part = 0
    return integer_part + fraction_part

def get_value_NUM_RATIONAL(num_string):
    NUM_FRACTION_match = re.fullmatch(NUM_FRACTION, num_string)
    if NUM_FRACTION_match:
        return get_value_NUM_FRACTION(num_string)
    NUM_INTEGER_PLUS_FRACTION_match = re.fullmatch(NUM_INTEGER_PLUS_FRACTION, num_string)
    if NUM_INTEGER_PLUS_FRACTION_match:
        return get_value_NUM_INTEGER_PLUS_FRACTION(num_string)
    return get_value_NUM_INTEGER(num_string)

def extract_numbers(text):
    extracted_numbers = []
    for match in re.finditer(NUM_RATIONAL, text):
        extracted_numbers.append({
            'marker': match.group(),
            'span': match.span(),
            'value': get_value_NUM_RATIONAL(match.group())
        })
    return extracted_numbers

print(extract_numbers("یی، یک دو سال"))

[{'marker': 'یک', 'span': (4, 6), 'value': 1}, {'marker': 'دو', 'span': (7, 9), 'value': 2}]


In [3]:
prefixes = {
"یوکتو":  'y',
"زپتو": 'z',
"آتو" : 'a',
"فمتو" : 'f',
"پیکو" : 'p',
"نانو" : 'n',
"میکرو" : 'u',
"میلی" : 'm',
"سانتی" : 'c',
"دسی" : 'd',
"دکا" : 'da',
"هکتو" : 'h',
"کیلو" : 'k',
"مگا" : 'M',
"گیگا" : 'G',
"ترا" : 'T',
"پتا" : 'P',
"اگزا" : 'E',
"زتا" : 'Z',
"یوتا" : 'Y',
"کیبی" : 'Ki',
"مبی" : 'Mi',
"گیبی" : 'Gi',
"تبی" : 'Ti',
"پبی" : 'Pi',
"اگزبی" : 'Ei',
"زبی" : 'Zi',
"یوبی" : 'Yi',

'ک.': 'k',
'م.': 'm',


'yocto': 'y',
'zepto': 'z',
'atto': 'a',
'femto': 'f',
'pico': 'p',
'nano': 'n',
'micro': 'u',
'µ': 'u',
'milli': 'm',
'centi': 'c',
'deci': 'd',
'deca': 'da',
'hecto': 'h',
'kilo': 'k',
'mega': 'M',
'giga': 'G',
'tera': 'T',
'peta': 'P',
'exa': 'E',
'zetta': 'Z',
'yotta': 'Y',
"kibi" : 'Ki',
"mebi" : 'Mi',
"gibi" : 'Gi',
"tebi" : 'Ti',
"pebi" : 'Pi',
"exbi" : 'Ei',
"zebi" : 'Zi',
"yobi" : 'Yi',
}

temp = list(prefixes.values())
for val in temp:
    prefixes[val] = val

with open('prefixes.json', 'w', encoding='utf-8') as fp:
    json.dump(prefixes, fp, indent=4)

prefixes

[{'marker': 'هشتاد و پنج صدم', 'span': (1, 16), 'value': 0.85}]

In [4]:
import json

units = {'%': '%',
 'BTU': 'BTU',
 'British thermal units': 'British thermal units',
 'Cal': 'Cal',
 'Calories': 'Calories',
 'Earth masses': 'Earth masses',
 'Hz': 'Hz',
 'Imp. fl oz': 'Imp. fl oz',
 'Imp. fluid ounces': 'Imp. fluid ounces',
 'Imp. gal': 'Imp. gal',
 'Imp. gallons': 'Imp. gallons',
 'K': 'K',
 'L/100 km': 'L/100 km',
 'L/100km': 'L/100km',
 'MP': 'megapixels',
 'Pa': 'Pa',
 'US therm': 'US therm',
 'US therms': 'US therms',
 'acre ft': 'acre ft',
 'acre-feet': 'acre-feet',
 'acres': 'acres',
 'amperes': 'amperes',
 'amps': 'amps',
 'arcmins': 'arcmins',
 'arcminutes': 'arcminutes',
 'arcseconds': 'arcseconds',
 'arcsecs': 'arcsecs',
 'astronomical units': 'astronomical units',
 'atm': 'atm',
 'atmospheres': 'atmospheres',
 'au': 'au',
 'bar': 'bar',
 'barrel': 'barrel',
 'barrels': 'barrels',
 'bars': 'bars',
 'bit': 'bit',
 'bits': 'bits',
 'bu': 'bushels',
 'bushels': 'bushels',
 'byte': 'byte',
 'bytes': 'bytes',
 'c': 'c',
 'cal': 'cal',
 'calories': 'calories',
 'carats': 'carats',
 'centuries': 'centuries',
 'cm': 'cm',
 'cm²': 'cm²',
 'cm³': 'cm³',
 'cc': 'cm³',
 'cubic centimeters': 'cubic centimeters',
 'cubic feet': 'cubic feet',
 'cubic inches': 'cubic inches',
 'cubic kilometers': 'cubic kilometers',
 'cubic meters': 'cubic meters',
 'cubic miles': 'cubic miles',
 'cubic yards': 'cubic yards',
 'cups': 'cups',
 'daltons': 'daltons',
 'day': 'day',
 'days': 'days',
 'dec': 'dec',
 'decades': 'decades',
 'deg': 'deg',
 'deg. C': 'deg. C',
 'deg. F': 'deg. F',
 'degrees': 'degrees',
 'degrees Celsius': 'degrees Celsius',
 'degrees Fahrenheit': 'degrees Fahrenheit',
 'dots per centimeter': 'dots per centimeter',
 'dots per inch': 'dots per inch',
 'dpcm': 'dpcm',
 'dpi': 'dpi',
 'dunams': 'dunams',
 'electronvolt': 'electronvolt',
 'electronvolts': 'electronvolts',
 'em': 'em',
 'fathoms': 'fathoms',
 'feet': 'feet',
 'feet³': 'feet³',
 'fl oz': 'fl oz',
 'fluid ounces': 'fluid ounces',
 'fm': 'fathoms',
 'ft': 'ft',
 'fur': 'furlongs',
 'furlongs': 'furlongs',
 'g-force': 'g-force',
 'gal': 'gal',
 'gallons': 'gallons',
 'gram': 'gram',
 'grams': 'grams',
 'hL': 'hL',
 'hectares': 'hectares',
 'hertz': 'hertz',
 'horsepower': 'horsepower',
 'hour': 'hour',
 'hours': 'hours',
 'hp': 'hp',
 'in': 'in',
 'inHg': 'inHg',
 'inches': 'inches',
 'inches of mercury': 'inches of mercury',
 'inches²': 'inches²',
 'inches³': 'inches³',
 'joules': 'joules',
 'kW-hour': 'kW-hour',
 'karats': 'karats',
 'kelvins': 'kelvins',
 'kilowatt-hours': 'kilowatt-hours',
 'kn': 'kn',
 'knots': 'knots',
 'lb': 'lb',
 'lbf⋅ft': 'lbf⋅ft',
 'light years': 'light years',
 'light yrs': 'light yrs',
 'liter': 'liter',
 'liters': 'liters',
 'liters per 100 kilometers': 'liters per 100 kilometers',
 'liters per kilometer': 'liters per kilometer',
 'lux': 'lux',
 'ly': 'ly',
 'meters': 'meters',
 'metric cups': 'metric cups',
 'metric pints': 'metric pints',
 'metric tons': 'metric tons',
 'mi': 'mi',
 'mile-scandinavian': 'mile-scandinavian',
 'miles': 'miles',
 'miles per Imp. gallon': 'miles per Imp. gallon',
 'miles per gallon': 'miles per gallon',
 'miles/gal': 'miles/gal',
 'miles/gal Imp.': 'miles/gal Imp.',
 'millimeters of mercury': 'millimeters of mercury',
 'min': 'min',
 'mins': 'mins',
 'minutes': 'minutes',
 'mi³': 'mi³',
 'mmHg': 'mmHg',
 'mole': 'mole',
 'moles': 'moles',
 'month': 'month',
 'months': 'months',
 'mpt': 'mpt',
 'm': 'm',
 'm³': 'm³',
 'nautical miles': 'nautical miles',
 'newton': 'newton',
 'newton-meters': 'newton-meters',
 'newtons': 'newtons',
 'nmi': 'nmi',
 'ohms': 'ohms',
 'ounces': 'ounces',
 'oz': 'oz',
 'oz troy': 'oz troy',
 'parsec': 'parsec',
 'parsecs': 'parsecs',
 'parts per million': 'parts per million',
 'ppm': 'parts per million',
 'parts/million': 'parts/million',
 'pascals': 'pascals',
 'percent': 'percent',
 'permille': 'permille',
 'permyriad': 'permyriad',
 'pints': 'pints',
 'pixels': 'pixels',
 'points': 'points',
 'pound-feet': 'pound-feet',
 'pound-force': 'pound-force',
 'pounds': 'pounds',
 'pounds of force': 'pounds of force',
 'ppcm': 'ppcm',
 'ppi': 'ppi',
 'psi': 'psi',
 'px': 'pixels',
 'qts': 'qts',
 'quarts': 'quarts',
 'rad': 'rad',
 'radians': 'radians',
 'rev': 'rev',
 'revolution': 'revolution',
 's':  's',
 'sec': 'sec',
 'seconds': 'seconds',
 'secs': 'secs',
 'smi': 'smi',
 'solar luminosities': 'solar luminosities',
 'solar masses': 'solar masses',
 'solar radii': 'solar radii',
 'sq feet': 'sq feet',
 'sq miles': 'sq miles',
 'square centimeters': 'square centimeters',
 'square feet': 'square feet',
 'square inches': 'square inches',
 'square kilometers': 'square kilometers',
 'square meters': 'square meters',
 'square miles': 'square miles',
 'square yards': 'square yards',
 'st': 'stones',
 'stone': 'stone',
 'stones': 'stones',
 't': 't',
 'tablespoons': 'tablespoons',
 'tbsp': 'tbsp',
 'teaspoons': 'teaspoons',
 'ton': 'ton',
 'tons': 'tons',
 'troy ounces': 'troy ounces',
 'tsp': 'tsp',
 'volts': 'volts',
 'watts': 'watts',
 'weeks': 'weeks',
 'wk': 'wk',
 'yards': 'yards',
 'yards²': 'yards²',
 'yards³': 'yards³',
 'yd': 'yd',
 'yd²': 'yd²',
 'years': 'years',
 'yr': 'yr',
 '°': '°',
 '°C': '°C',
 '°F': '°F',
 'آمپر': 'amps',
 'اتمسفر': 'atm',
 'اسب بخار': 'hp',
 'الکترون ولت': 'electronvolts',
 'الکترون\u200cولت': 'electronvolt',
 'اهم': 'ohms',
 'اونس': 'oz',
 'اونس تروا': 'oz troy',
 'اونس سیال': 'fl oz',
 'اونس سیال انگلیسی': 'Imp. fl oz',
 'اینچ': 'inches',
 'اینچ جیوه': 'inHg',
 'اینچ مربع': 'inches²',
 'اینچ مکعب': 'inches³',
 'بار': 'bar',
 'بایت': 'byte',
 'بخش در میلیون': 'parts per million',
 'بخش/میلیون': 'parts/million',
 'بشکه': 'barrel',
 'بیت': 'bit',
 'بی\u200cتی\u200cیو': 'BTU',
 'تابندگی خورشید': 'solar luminosities',
 'ترم آمریکایی': 'US therm',
 'تن': 'tons',
 'تن متریک': 't',
 'ثانیه': 'secs',
 'ثانیهٔ قوسی': 'arcsecs',
 'جرم خورشید': 'solar masses',
 'جرم زمین': 'Earth masses',
 'جریب': 'acres',
 'جریب فوت': 'acre ft',
 'دالتون': 'daltons',
 'درجه': 'degrees',
 'درجهٔ سلسیوس': 'deg. C',
 'درجهٔ فارنهایت': 'deg. F',
 'درجهٔ کلوین': 'K',
 'درصد': 'percent',
 'دقیقه': 'mins',
 'دقیقهٔ قوسی': 'arcmins',
 'دهه': 'dec',
 'ده\u200cهزارم': 'permyriad',
 'دور': 'rev',
 'دونوم': 'dunams',
 'رادیان': 'radians',
 'روز': 'days',
 'ساعت': 'hours',
 'سال': 'years',
 'سال نوری': 'light yrs',
 'سانت': 'cm',
 'سده': 'c',
 'سنگ': 'stone',
 'شعاع خورشید': 'solar radii',
 'عیار': 'karats',
 'فوت': 'feet',
 'فوت مربع': 'sq feet',
 'فوت مکعب': 'feet³',
 'قاشق غ.': 'tbsp',
 'قاشق غذاخوری': 'tablespoons',
 'قاشق چ.': 'tsp',
 'قاشق چای\u200cخوری': 'teaspoons',
 'قیراط': 'carats',
 'لوکس': 'lux',
 'لیتر': 'liters',
 'لیتر در کیلومتر': 'liters per kilometer',
 'لیتر در ۱۰۰ کیلومتر': 'liters per 100 kilometers',
 'لیتر/کیلومتر': 'liters/km',
 'لیتر/۱۰۰ کیلومتر': 'L/100 km',
 'ماه': 'months',
 'مایل': 'miles',
 'مایل اسکاندیناوی': 'smi',
 'مایل در امپریال گالن': 'miles per Imp. gallon',
 'مایل در ساعت': 'miles/hour',
 'مایل در گالن': 'miles per gallon',
 'مایل دریایی': 'nmi',
 'مایل مربع': 'sq miles',
 'مایل مکعب': 'mi³',
 'مایل/امپریال گالن': 'miles/gal Imp.',
 'مایل/گالن': 'miles/gal',
 'متر': 'm',
 'متر در ثانیه': 'meters/sec',
 'مول': 'mole',
 'مگاپیکسل': 'megapixels',
 'میلی\u200cمتر جیوه': 'mmHg',
 'میلی\u200cمول در لیتر': 'millimoles per liter',
 'میلی\u200cگرم در دسی\u200cلیتر': 'mg/dL',
 'م\u200cم': 'mm',
 'نقطه در اینچ': 'dots per inch',
 'نقطه در سانتی\u200cمتر': 'dots per centimeter',
 'نیروی جاذبه': 'g-force',
 'نیوتن': 'newton',
 'نیوتون': 'newton',
 'نیوتن\u200cمتر': 'N⋅m',
 'هرتز': 'Hz',
 'هزارم': 'permille',
 'هفته': 'weeks',
 'هکتار': 'hectares',
 'وات': 'watts',
 'واحد نجومی': 'au',
 'ولت': 'volts',
 '٪': '%',
 'پارسک': 'parsecs',
 'پاسکال': 'pascals',
 'پاینت': 'pints',
 'پاینت متریک': 'mpt',
 'پونت': 'points',
 'پوند': 'pounds',
 'پوند در اینچ مربع': 'psi',
 'پوند-فوت': 'lbf⋅ft',
 'پوند-نیرو': 'pound-force',
 'پیمانه': 'cups',
 'پیمانهٔ متریک': 'mcup',
 'پیکسل': 'pixels',
 'پیکسل در اینچ': 'pixels per inch',
 'پیکسل در سانتی\u200cمتر': 'pixels per centimeter',
 'پیکومتر': 'pm',
 'ژول': 'joules',
 'ک.وات\u200cساعت': 'kW-hour',
 'کالری': 'Cal',
 'کوارت': 'qts',
 'کیلومتر در ساعت': 'km/hour',
 'کیلووات\u200cساعت': 'kilowatt-hours',
 'گالن': 'gal',
 'گالن امپریال': 'Imp. gal',
 'گرم': 'grams',
 'گره': 'kn',
 'یارد': 'yards',
 '\u200e°C': '°C',
 '\u200e°F': '°F',
 '″ Hg': '″ Hg'}
with open('units.json', 'w', encoding='utf-8') as fp:
    json.dump(units, fp, indent=4)

In [5]:
prefixes = {
"یوکتو":  'y',
"زپتو": 'z',
"آتو" : 'a',
"فمتو" : 'f',
"پیکو" : 'p',
"نانو" : 'n',
"میکرو" : 'u',
"میلی" : 'm',
"سانتی" : 'c',
"دسی" : 'd',
"دکا" : 'da',
"هکتو" : 'h',
"کیلو" : 'k',
"مگا" : 'M',
"گیگا" : 'G',
"ترا" : 'T',
"پتا" : 'P',
"اگزا" : 'E',
"زتا" : 'Z',
"یوتا" : 'Y',
"کیبی" : 'Ki',
"مبی" : 'Mi',
"گیبی" : 'Gi',
"تبی" : 'Ti',
"پبی" : 'Pi',
"اگزبی" : 'Ei',
"زبی" : 'Zi',
"یوبی" : 'Yi',

'ک.': 'k',
'م.': 'm',


'yocto': 'y',
'zepto': 'z',
'atto': 'a',
'femto': 'f',
'pico': 'p',
'nano': 'n',
'micro': 'u',
'µ': 'u',
'milli': 'm',
'centi': 'c',
'deci': 'd',
'deca': 'da',
'hecto': 'h',
'kilo': 'k',
'mega': 'M',
'giga': 'G',
'tera': 'T',
'peta': 'P',
'exa': 'E',
'zetta': 'Z',
'yotta': 'Y',
"kibi" : 'Ki',
"mebi" : 'Mi',
"gibi" : 'Gi',
"tebi" : 'Ti',
"pebi" : 'Pi',
"exbi" : 'Ei',
"zebi" : 'Zi',
"yobi" : 'Yi',
}

temp = list(prefixes.values())
for val in temp:
    prefixes[val] = val

with open('prefixes.json', 'w', encoding='utf-8') as fp:
    json.dump(prefixes, fp, indent=4)

prefixes

{'یوکتو': 'y',
 'زپتو': 'z',
 'آتو': 'a',
 'فمتو': 'f',
 'پیکو': 'p',
 'نانو': 'n',
 'میکرو': 'u',
 'میلی': 'm',
 'سانتی': 'c',
 'دسی': 'd',
 'دکا': 'da',
 'هکتو': 'h',
 'کیلو': 'k',
 'مگا': 'M',
 'گیگا': 'G',
 'ترا': 'T',
 'پتا': 'P',
 'اگزا': 'E',
 'زتا': 'Z',
 'یوتا': 'Y',
 'کیبی': 'Ki',
 'مبی': 'Mi',
 'گیبی': 'Gi',
 'تبی': 'Ti',
 'پبی': 'Pi',
 'اگزبی': 'Ei',
 'زبی': 'Zi',
 'یوبی': 'Yi',
 'ک.': 'k',
 'م.': 'm',
 'yocto': 'y',
 'zepto': 'z',
 'atto': 'a',
 'femto': 'f',
 'pico': 'p',
 'nano': 'n',
 'micro': 'u',
 'µ': 'u',
 'milli': 'm',
 'centi': 'c',
 'deci': 'd',
 'deca': 'da',
 'hecto': 'h',
 'kilo': 'k',
 'mega': 'M',
 'giga': 'G',
 'tera': 'T',
 'peta': 'P',
 'exa': 'E',
 'zetta': 'Z',
 'yotta': 'Y',
 'kibi': 'Ki',
 'mebi': 'Mi',
 'gibi': 'Gi',
 'tebi': 'Ti',
 'pebi': 'Pi',
 'exbi': 'Ei',
 'zebi': 'Zi',
 'yobi': 'Yi',
 'y': 'y',
 'z': 'z',
 'a': 'a',
 'f': 'f',
 'p': 'p',
 'n': 'n',
 'u': 'u',
 'm': 'm',
 'c': 'c',
 'd': 'd',
 'da': 'da',
 'h': 'h',
 'k': 'k',
 'M': 'M',
 'G': 

In [6]:
import json

units = {}
prefixes = {}

with open('units.json', 'r', encoding='utf-8') as fp:
    units = json.load(fp)

with open('prefixes.json', 'r', encoding='utf-8') as fp:
    prefixes = json.load(fp)

In [7]:
import pint

ureg = pint.UnitRegistry()

In [13]:
from unit_extractor import extract_units, unit_to_quantity


In [9]:
# item-> number -> unit | number -> unit -> item |

In [15]:
import pint
import json

a = extract_units(r'متر بر مجذور ثانیه')
print(unit_to_quantity(a[0]['object']))


['شتاب']
